In [2]:
%pip install SciencePlots

Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scienceplots
plt.style.use(['science', 'nature','no-latex'])

from hamiltonians import Hamiltonian
from qnute_params import QNUTE_params as Params
from qnute_output import QNUTE_output as Output
from qnute import qnute

# Figures for the paper
## 1. Fidelity to Normalized Time Evolution
This series of figures should explore the fidelity of the simulated time evolution compared to the normalized time evolution (with Taylor series). Each graph should cover a large number of randomized $k$-local Hamiltonians. The lines on the graph show the mean fidelity of the state at time t in the evolution, showing standard deviations at fixed intervals. Different lines correspond to choosing different parameters `D`, `dt`, `trotter_flag`, `taylor_truncate_a`, `taylor_truncate_h`.
For a given qubit lattice and locality $k$, a uniformly random unit-length Hamiltonian is chosen by taking randomly generated $X_I \sim \mathcal{N}(0,1)$ sampled from the standard normal distribution for each $\hat{\sigma}_I$ that can contribute to a $k$-local Hamiltonian, and normalizing the random vector $Y_I = \frac{X_I}{\sum_I X_I^2}$, and lastly generating uniform random phases $\Phi_I \sim U(0,2\pi)$ giving the coefficients $e^{i\Phi_I}Y_I$.

The graphs will be Fidelity (y-axis) vs Simulation time (x-axis) line graphs with different colors corresponding to different parameters chosen.

## 2. Fidelity variation due to measurement samples
This series of figures shows how changing the number of measurement samples in the QNUTE simulation to generate the system of linear equations in each Trotter step affects the fidelity of the simulation when compared to taking the theoretical expected values with knowledge of the state vector.

As before, each plot corresponds to a different Hamiltonian structure. We will also separate plots by the different run parameters to see if they affect the fidelity.

These should be mulit-bar graphs with error bars. Each set of bars corresponds to a number of measurement samples and the different bars in the set indicate the different run parameters, along with an aggregate over all the experiments.

Fidelity (y-axis) vs Number of Samples (x-axis, log scale)

### Generating Random k-local Hamiltonians
Ideas for how to find all possible connected paths of length $k$ in the lattice of $N$ points.
1. Loop through all possibilities of $k$ points and see if they form a connected path. $O(N^k)$ Very inneficient.
2. Loop through every point and recursively build all possible paths of length k starting from that point. $O(N2^{d(k-1)})$

Coordinate ordering by index i.e. $(0,0)\prec(0,1)\prec(1,0)\prec(1,1)$

In [11]:
a = [ (1,1), (0,1), (0,0), (1,0) ]
b = sorted(a)
tb = tuple(b)
c = ((0, 0), (0, 1), (1, 0), (1, 1))

tb == c

True

In [27]:
(0,1) + (1,2)

(0, 1, 1, 2)

In [5]:
from helpers import *

In an unbounded grid, there will be a total of $1\cdot2^d\cdot(2^d-1)^{k-1} = O(2^{dk})$ paths of length $k>0$ originating from a fixed starting point that don't self intersect. A path of length $k$ has $k+1$ vertices in it.

In [241]:
def get_paths(start, depth,d,l, illegal_dir=None):
    '''
    returns all paths of depth=depth originating from the point start in the l^d lattice using depth first search
    the paths are sorted coordinate wise
    '''
    if type(start) is not tuple:
        start = (start, )
    assert len(start) == d, 'start must be a d-dimensional tuple'
    
    if depth == 0:
        return [[start]]  if d > 1 else [[start[0]]]
    
    paths = []
    for i in range(d):
        for j in range(2):
            if illegal_dir == (i,(-1)**j):
                continue
            n_node = list(start)
            n_node[i] += (-1)**j
            n_node = tuple(n_node)
            if in_lattice(n_node,d,l):
                n_paths = get_paths(n_node, depth-1, d,l, illegal_dir=(i,(-1)**(j+1)))
                for path in n_paths:
#                     paths.append( [start]+path )
                    paths.append( [start if d > 1 else start[0]]+path )
                    
    return paths

In [233]:
start = 2
k=1
d=1
l=5
a = get_paths(start,k,d,l)
a

[[2, 3], [2, 1]]

In [237]:
tuple(3)

TypeError: 'int' object is not iterable

In [242]:
def get_k_local_domains(k,d,l):
    domains = set()
    for i in range(l**d):
        point = tuple(int_to_base(i,l,d))
        for k_ in range(k):
            n_paths = get_paths(point,k_,d,l)
#             print('point:{}, k={}, paths={}'.format(point, k_, n_paths))
            for path in n_paths:
                s_path = sorted(path)
                domains.add(tuple(s_path))
    return sorted(list(domains))

In [249]:
a=get_k_local_domains(2,2,2)
a

[((0, 0),),
 ((0, 0), (0, 1)),
 ((0, 0), (1, 0)),
 ((0, 1),),
 ((0, 1), (1, 1)),
 ((1, 0),),
 ((1, 0), (1, 1)),
 ((1, 1),)]

In [217]:
len(a)

20

In [214]:
get_k_local_domains(2,1,3)

[((0,),), ((0,), (1,)), ((1,),), ((1,), (2,)), ((2,),)]

In [278]:
def get_random_k_local_hamiltonian(k, d, l, qubit_map):
    domains = get_k_local_domains(k, d, l)
#     if qubit_map is None:
#         assert d==1, 'Default qubit map only valid for d=1'
    
    hm_list = [ [ [], np.zeros(3**len(domain),dtype=complex), list(domain) ] for domain in domains ]
    num_coefficients = 0
    for hm in hm_list:
        ndomain = len(hm[2])
        num_coefficients += 3**ndomain
        for p in range(3**ndomain):
            pstring = int_to_base(p, 3, ndomain)
            for i in range(ndomain):
                pstring[i] += 1
            hm[0].append(base_to_int(pstring, 4))
    
    
    return hm_list

In [275]:
hm = [ [1,2,3], [], [1] ]
hm[1] = np.zeros(3,dtype=complex)
hm

[[1, 2, 3], array([0.+0.j, 0.+0.j, 0.+0.j]), [1]]

In [280]:
map2 = {(0,0):0,(0,1):1,(1,0):2,(1,1):1}

get_random_k_local_hamiltonian(2,1,2,None)

[[[1, 2, 3], array([0.+0.j, 0.+0.j, 0.+0.j]), [0]],
 [[5, 6, 7, 9, 10, 11, 13, 14, 15],
  array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j]),
  [0, 1]],
 [[1, 2, 3], array([0.+0.j, 0.+0.j, 0.+0.j]), [1]]]